# Funcion para hacer el scraping

In [ ]:
# BLOQUE 1: Importaciones y configuración inicial

import requests # para hacer peticiones HTTP (descargar paginas web)
from bs4 import BeautifulSoup # para analizar el HTML descargado
import sqlite3 
import pandas as pd 
import time 
import re # para expresiones regulares (extraer numeros de texto)
from urllib.parse import urljoin, quote # urljoin: Construir URLs completas a partir de relativas 
import json # para trabajar con JSON 

print("✅ Librerías importadas correctamente")
print("🕷️ Comenzando la infiltración en Books To Scrape...") 


# BLOQUE 2: Funciones de Web Scraping 

def get_all_categories ():
    ''' OBTIENE TODAS LAS CATEGORIAS DEL SITIO'''
    url = "https://books.toscrape.com" 
    respuesta = requests.get(url) # se descarga la web 
    soup = BeautifulSoup(respuesta.content, 'html.parser')  
    # convierte texto HTML ilegible, en un objeto inteligente que se pueda navegar 
    
    categories = []
    nav_list = soup.find('ul', class_='nav nav-list')
    if nav_list:
        category_links = nav_list.find_all ('a') [1:] # Saltar "Books"
        for link in category_links:
            category_name = link.text.strip()  # Extrae el texto del enlace y quita espacios 
            category_url = urljoin(url, link['href'])  # Convierte la URL relativa a absoluta 
            categories.append({ 
                'name' : category_name,
                'url' : category_url 
            })
            
    print (f"🎯 Encontradas {len(categories)} categorias")
    return categories 


def book_quantity (book_url): 
    ''' OBTIENE LA CANTIDAD EN STOCK DE UN LIBRO DESDE SU PAGINA DE DETALLES  '''
    try:
        soup_quantity = BeautifulSoup(requests.get(book_url).content,'html.parser') # descarga y parsea la pagina individual del libro 
        quantity_text = soup_quantity.select_one('p.instock.availability').get_text(strip=True) 
        # Busca el <p> con clases "instock" y "availability", extrae su texto limpio
        match = re.search(r'\((\d+)\)', quantity_text) # busca un patron para extraer el numero entero 
        if match:
            return int (match.group(1)) # devuelve la cantidad encontrada 
        else:
            return 0 # si no se encuentra la cantidad, devuelve 0 
    except Exception as e :
        print (f"❌ Erorr obteniendo cantidad para {book_url}:{e}")
        return 0 # en caso de error, devuelve 0 
    


def scrape_books_from_page(page_url): 
    ''' SCRAPE LIBROS DE UNA PAGINA ESPECIFICA  '''
    response = requests.get(page_url) # Descarga la pagina 
    soup = BeautifulSoup(response.content,'html.parser' ) # Parsea el HTML 
    
    books = []
    book_containers = soup.find_all ('article', class_ ='product_pod') 
    # cada libro esta dentro de un <article class="product_pod"> 
    
    for book in book_containers: 
        try:
            # TITULO
            title_element = book.find('h3').find('a') # busca la a dentro del h3 
            title = title_element['title'] # el titulo completo esta en el atributo "title"

            # URL DEL LIBRO PARA MAS DETALLES 
            book_url = urljoin(page_url, title_element['href']) 
            # cosntruye la url absoluta del libro a partir de su href relativo
            
            
            # PRECIO
            price_element = book.find('p', class_= 'price_color') 
            price_text = price_element.text.strip() if price_element else "£0.00"
            price = float (price_text.lstrip('Â£')) # Elimina los caracteres "Â" y "£" del inicio y convierte a número decimal
            
            # RATING 
            rating_element = book.find ('p', class_= 'star-rating')
            rating_class = rating_element['class'][1] if rating_element else 'Zero' 
            # La clase CSS indica el rating
            rating_map = {'One': 1, 'Two' : 2 , 'Three': 3, 'Four': 4, 'Five': 5, "Zero" : 0 } 
            rating = rating_map.get(rating_class, 0) 
            # Convierte la palabra en inglés a número 
            
            # STOCK 
            stock_element = book.find ('p', class_= 'instock availability')
            in_stock = 'In stock' in stock_element.text if stock_element else False 
            # Verifica si el texto contiene "In stock" → True/False
            quantity = book_quantity(book_url)
            # Llama a la función anterior para obtener la cantidad exacta en stock
            # ⚠️ NOTA: Esto hace una petición HTTP extra POR CADA LIBRO (lento)
            
            books.append({ # se agrega todos los datos del libro al diccionario 
                
                'title': title,
                'price': price,
                'rating' : rating,
                'in_stock' : in_stock,
                'quantity': quantity,
                'url': book_url
            })
            
        except Exception as e:
            print(f"❌ Error procesando libro: {e}") 
            continue
    return books 

def scrape_all_books (): 
    ''' SCRAPE TODOS LOS LIBROS DEL SITIO '''
    all_books = []
    categories = get_all_categories() # obtiene la lista de categorias
    for i, category in enumerate(categories): 
        print (f"Procesando categoria {i+1}/ {len(categories)}: {category['name']}")
        
        page_num = 1 
        current_url = category['url'] # URL de la primera pagina de la categoria 
        
        while current_url: # sigue el bucle mientras haya paginas
            print (f" Pagina {page_num}")
            books_on_page = scrape_books_from_page(current_url) 
            # Extare todos los libros de la pagina actual 
            
            for book in books_on_page:
                book['category'] = category['name']
                # agrega el nombre de la categoria a cada libro 
                
            all_books.extend(books_on_page) # se agrega los libros a la lista total 
            
            # BUSCAR SIGUIENTE PAGINA 
            response = requests.get(current_url)
            soup = BeautifulSoup(response.content, 'html.parser')
            next_button = soup.find('li', class_= 'next')
            # busca el boton next para ir a la siguiente pagina 
            
            if next_button and next_button.find('a'):
                next_url = next_button.find('a')['href']
                current_url = urljoin(current_url, next_url)
                # construye la URL a la siguiente pagina 
                page_num += 1 
            else:
                current_url = None # no hay mas paginas y sale del while 
                
            time.sleep (0.5) # ser amigables con el servidor, espera 0.5 entre peticiones 
        
    print (f" 🎉 Scraping completado: {len(all_books)} libros encontrados")
    return all_books


# BLOQUE 3: EJECUTAR EL SCRAPING 
books_data = scrape_all_books

# Mostrar muestra de datos 
print ("\n Muestra de los primeros 3 libros: ")
for i, book in enumerate(books_data[:3]):
    print(f"{i+1}. {book['title']} - {book['price']} - ⭐{book['rating']} - {book['category']} - {book['quantity']}")
    # se imprime los primeros 3 libros como muestra 